In [1]:
import pandas as pd
import numpy as np

In [2]:
root = '/home/sihartist/Desktop/'

In order to display all columns

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_excel(root + 'fraud-detection/dataset/clean_data.xlsx', dtype=str)

## Analysis

### Attribute names

In [46]:
attr = pd.read_excel(root + 'fraud-detection/dataset/attributes.xlsx', dtype=str)

In [76]:
attr

,PROCESSING_CODE CHAR(2 BYTE),V1
0,FUNCTION_CODE CHAR(3 BYTE),V2
1,NETWORK_CODE CHAR(2 BYTE),V3
2,ISSUING_BANK CHAR(6 BYTE),V4
3,SERVICE_CODE CHAR(3 BYTE),V5
4,START_EXPIRY_DATE DATE,V6
5,END_EXPIRY_DATE DATE,V7
6,TRANSMISSION_DATE_AND_TIME DATE,V8
7,"TRANSACTION_AMOUNT NUMBER(18,3)",V9
8,"BILLING_AMOUNT NUMBER(18,3)",V10
9,"TRANSACTION_FEE NUMBER(18,3)",V11


### Analysis

In [5]:
df.head(2)

,V1,V2,V3,V4,V6,V7,V8,V9,V10,V12,V13,V16,V19,V21,V22,V23,V24,V25,V26,V27,CLASS
0,31,281,22,010112,2009-02-07 21:21:42,2009-12-26 00:00:00,02/07/2009 21:21:42,285328,33443,010112,6011,21140121124C,071181,01000002,01000002,26/12/2009,573908,952,384,VERSUS BANK 2 PLATEAUX REGION LAGUNECI,0
1,31,281,22,010112,2009-06-26 11:16:47,2009-12-26 00:00:00,26/06/2009 11:16:47,483993,652487,010112,6011,21140121124C,004737,01000001,01000001,26/12/2009,695410,952,384,VERSUS BANK PLATEAU REGION LAGUNECI,0


In [6]:
df.tail(2)

,V1,V2,V3,V4,V6,V7,V8,V9,V10,V12,V13,V16,V19,V21,V22,V23,V24,V25,V26,V27,CLASS
140652,01,281,22,010112,2010-02-10 10:30:04,2010-12-26 00:00:00,02/10/2010 10:30:04,769069,798416,010112,6011,21140121124C,016736,00000008,01000003,26/12/2010,708569,952,192,VERSUS BANK ZONE 4 REGION LAGUNECI,1
140653,01,281,22,010112,2010-09-27 14:53:19,2010-12-26 00:00:00,27/09/2010 14:53:19,797365,82712,010112,6011,21140121124C,036370,01000001,01000001,26/12/2010,640035,952,360,VERSUS BANK PLATEAU REGION LAGUNECI,1


In [7]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140654 entries, 0 to 140653
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   V1      140654 non-null  object
 1   V2      140654 non-null  object
 2   V3      140654 non-null  object
 3   V4      140654 non-null  object
 4   V6      140654 non-null  object
 5   V7      140654 non-null  object
 6   V8      140654 non-null  object
 7   V9      140654 non-null  object
 8   V10     140654 non-null  object
 9   V12     140654 non-null  object
 10  V13     140654 non-null  object
 11  V16     140654 non-null  object
 12  V19     140654 non-null  object
 13  V21     140654 non-null  object
 14  V22     140654 non-null  object
 15  V23     140654 non-null  object
 16  V24     140654 non-null  object
 17  V25     140654 non-null  object
 18  V26     140654 non-null  object
 19  V27     140654 non-null  object
 20  CLASS   140654 non-null  object
dtypes: object(21)
memory usage: 22.5+

In [8]:
df['V9'] = df['V9'].astype(float)
df['V10'] = df['V10'].astype(float)
df['V24'] = df['V24'].astype(float)
df['CLASS'] = df['CLASS'].astype(int)

In [9]:
n_fraud = df.loc[df['CLASS']==0]
fraud = df.loc[df['CLASS']==1]

In [14]:
original = pd.read_excel(root + 'fraud-detection/dataset/Authorization_data_v2.xlsx', dtype=str)

In [12]:
amount = df.filter(['V9', 'V10', 'V24'])
amount.head()

,V9,V10,V24
0,285328.0,33443.0,573908.0
1,483993.0,652487.0,695410.0
2,713040.0,554381.0,446081.0
3,592014.0,778437.0,460422.0
4,18805.0,314278.0,211343.0
